In [ ]:

from chartist.chart import Chartist
from IPython.display import Markdown as md

# Intro

Below is an example workflow using the Chartist cli

Say you have a chart displaying loss curves and evaluation metrics in the
README of a model's repo. For the purposes of the demo we'll keep the Markdown
in a string variable so we can operate on it, but IRL you'd probs read the file 
into memory as some point...or not, your choice, I'm not your mum.

Note below:

  - The existing image tags have dummy src urls
  - the `alt` text is what Chartist uses to match when updating charts

In [ ]:
endpoint = 'https://yamgal-server-c6l3dwv2sq-de.a.run.app'
demo_md = 'demo.md'
data_yaml = 'data.yaml'
config_yaml = 'config.yaml'

In [ ]:
%%writefile {demo_md}
# My Model

It draws boxes around shit! It draws them good. You could use Markdown or HTML
style image insert tags, we'll use HTML because this allows us to resize.

# Training History

<img src="http://something:...." alt="training_loss" height="500" width="500" >

The end

You train your model and end up with a training curve and a metric value

In [ ]:
%%writefile {data_yaml}
training_loss:
  - 0.9
  - 0.7
  - 0.3
  - 0.1
  - 0.02
  - 0.01

In [ ]:
%%writefile {config_yaml}
title: Training Logs
style: neon

We can generate a Chartist url using `yamgal.make_chartist_string` and insert them into the markdown text using `yamgal.find_and_replace_chart`

In [ ]:
import ruamel.yaml as yaml

def load_data(data_path):
    with open(data_path, 'r') as f:
        chart_data = yaml.round_trip_load(f)
    return chart_data
    
chart = Chartist.from_data_file('line',
                                data_yaml,
                                load_data,
                                config_path=config_yaml,
                                endpoint=endpoint
                               )


We can now insert this into the Markdown document.

In [ ]:
alt_text = 'training_loss'

with open(demo_md, 'r') as f:
    text = f.read()
    
new_text = chart.insert_into_text(text, alt_text)
print(new_text)

In [ ]:
display(md(new_text))

Next we do another training run and get another loss curve. We can add this to
the existing chart in the Markdown

In [ ]:
%%writefile {data_yaml}
training_loss:
  - 0.9
  - 0.6
  - 0.1
  - 0.05
  - 0.01
  - 0.01

In [ ]:
new_data = load_data(data_yaml)

# load existing chart from text
chart = Chartist.from_text(new_text, alt_text)
chart.add_ranges(new_data)
new_text = chart.insert_into_text(new_text, alt_text)
print(new_text)

In [ ]:
display(md(new_text))

You can also just append points to ranges

In [ ]:
%%writefile {data_yaml}
training_loss_00:
  - 0.01
  - 0.01
training_loss_01:
  - 0.1
  - 0.2

In [ ]:
new_points = load_data(data_yaml)

chart = Chartist.from_text(new_text, alt_text)
chart.append_to_ranges(new_points)
new_text = chart.insert_into_text(new_text, alt_text)
print(new_text)

In [ ]:
display(md(new_text))